In [1]:
from pyiem.network import Table as NetworkTable
from pyiem.util import get_dbconn, get_sqlalchemy_conn
import pandas as pd

MESOSITE = get_dbconn('mesosite')
mcursor = MESOSITE.cursor()

networks = []
mcursor.execute("""SELECT distinct network from stations where country = 'US' and 
 network ~* 'ASOS' and length(id) < 4""")
for row in mcursor:
    networks.append(row[0])
MESOSITE.close()

nt = NetworkTable(networks)
rows = []
sids = list(nt.sts.keys())
sids.sort()
for sid in sids:
    # Require stations to have data back to 1973
    if nt.sts[sid]['archive_begin'] is None or nt.sts[sid]['archive_begin'].year > 1973:
        continue

    # Figure out what our archive coverage is, 50 years would be perfect
    with get_sqlalchemy_conn("iem") as conn:
        df = pd.read_sql(
            "SELECT extract(year from day) as year, count(*) from summary where iemid = %s and "
            "max_feel is not null and day >= '1973-01-01' GROUP by year ORDER by year asc",
            conn,
            params=(nt.sts[sid]['iemid'], ),
            index_col='year',
        )
    years_with_data = len(df.loc[df['count'] > 300].index) 
    if years_with_data < 40:
        print(f'Skipping {sid}, due to only have {years_with_data}/50 years of data')
        continue
        
    # Get data
    with get_sqlalchemy_conn("asos") as conn:
        df = pd.read_sql(
            """WITH data as
                (SELECT distinct date_trunc('hour', valid) as ts from alldata WHERE station = %s
                and feel >= 99.5 and to_char(valid, 'mmdd') < '0721' and valid > '1973-01-01')
            select extract(year from ts) as year, count(*) from data GROUP by year ORDER by year asc""",
            conn,
            params=(sid, ),
            index_col="year",
        )
    df = df.reindex(range(1973, 2023)).fillna(0)
    rows.append({
        "sid": sid,
        "lat": nt.sts[sid]['lat'],
        "lon": nt.sts[sid]['lon'],
        "d2022": df.at[2022, "count"],
        "climate": df["count"].mean(),
        "network": nt.sts[sid]['network'],
        "d2022_rank": df.rank().at[2022, "count"]
    })
    print(f"{sid} d2022:{df.at[2022, 'count']} #{df.rank().at[2022, 'count']}, climate: {df['count'].mean()}")

df = pd.DataFrame(rows)

Skipping 1V4, due to only have 14/50 years of data
Skipping 3A1, due to only have 13/50 years of data
Skipping 3A6, due to only have 7/50 years of data
Skipping 3DU, due to only have 22/50 years of data
Skipping 42J, due to only have 10/50 years of data
Skipping 4BL, due to only have 14/50 years of data
Skipping 4CR, due to only have 3/50 years of data
Skipping 4HV, due to only have 14/50 years of data
Skipping 4MR, due to only have 7/50 years of data
Skipping 4SL, due to only have 3/50 years of data
Skipping 79J, due to only have 19/50 years of data
AAF d2022:101.0 #40.0, climate: 59.4
ABE d2022:0.0 #10.0, climate: 6.64
ABI d2022:191.0 #50.0, climate: 32.94
ABQ d2022:0.0 #25.0, climate: 0.02
ABR d2022:26.0 #47.0, climate: 10.66
ABY d2022:99 #35.5, climate: 72.02
Skipping ACB, due to only have 35/50 years of data
ACK d2022:0 #25.5, climate: 0.0
ACT d2022:316 #49.0, climate: 125.26
ACV d2022:0.0 #24.5, climate: 0.06
ACY d2022:1.0 #11.0, climate: 12.58
ADM d2022:172.0 #42.0, climate: 102

CYS d2022:0 #25.5, climate: 0.0
Skipping CZK, due to only have 3/50 years of data
Skipping CZZ, due to only have 22/50 years of data
DAA d2022:8.0 #11.0, climate: 32.84
DAB d2022:72.0 #42.0, climate: 45.82
DAG d2022:73.0 #27.0, climate: 78.78
DAL d2022:267 #45.0, climate: 112.18
DAN d2022:8.0 #21.5, climate: 17.9
DAY d2022:22.0 #46.0, climate: 7.04
DBQ d2022:22.0 #46.0, climate: 6.82
DCA d2022:5.0 #8.5, climate: 29.54
DDC d2022:66.0 #48.0, climate: 23.5
DEC d2022:61.0 #45.0, climate: 25.76
DET d2022:9.0 #38.0, climate: 6.54
Skipping DEW, due to only have 23/50 years of data
DFW d2022:184 #46.0, climate: 103.96
DHN d2022:112.0 #45.0, climate: 55.3
DHT d2022:6.0 #45.5, climate: 1.6
DIK d2022:2.0 #41.0, climate: 1.08
Skipping DKB, due to only have 21/50 years of data
Skipping DKK, due to only have 25/50 years of data
DLF d2022:133.0 #29.0, climate: 124.42
DLH d2022:0.0 #24.0, climate: 0.2
DLN d2022:0 #25.5, climate: 0.0
DLS d2022:0.0 #14.5, climate: 3.78
DMA d2022:37.0 #31.0, climate: 34.

HVN d2022:3.0 #45.0, climate: 1.14
HVR d2022:0.0 #22.5, climate: 0.5
Skipping HWD, due to only have 35/50 years of data
HYA d2022:0.0 #21.0, climate: 0.76
IAB d2022:85.0 #44.0, climate: 47.7
IAD d2022:3.0 #15.5, climate: 14.42
IAG d2022:1.0 #39.0, climate: 1.3
IAH d2022:224 #45.0, climate: 120.96
ICT d2022:80.0 #37.0, climate: 57.98
IDA d2022:0.0 #25.0, climate: 0.02
IGM d2022:17.0 #43.0, climate: 7.78
IKW d2022:2.0 #27.5, climate: 6.16
Skipping ILE, due to only have 34/50 years of data
ILG d2022:0.0 #6.0, climate: 13.96
ILM d2022:73 #39.0, climate: 51.3
Skipping ILN, due to only have 23/50 years of data
IML d2022:30.0 #50.0, climate: 4.5
IMT d2022:7.0 #42.0, climate: 2.98
IND d2022:24.0 #41.0, climate: 12.78
INK d2022:8.0 #11.0, climate: 38.28
INL d2022:0.0 #20.5, climate: 0.56
Skipping INS, due to only have 14/50 years of data
INT d2022:12.0 #43.5, climate: 5.86
INW d2022:0.0 #24.0, climate: 0.12
IPL d2022:276 #37.0, climate: 238.74
IPT d2022:0.0 #13.0, climate: 4.34
IRK d2022:26.0 #

NKX d2022:0.0 #23.5, climate: 0.2
NLC d2022:44 #31.5, climate: 40.7
NMM d2022:166 #46.5, climate: 70.98
NOG d2022:145.0 #13.0, climate: 212.18
Skipping NOW, due to only have 6/50 years of data
NPA d2022:104 #35.0, climate: 72.98
NQA d2022:168.0 #39.0, climate: 115.36
NQI d2022:429 #46.0, climate: 249.22
NQX d2022:619.0 #50.0, climate: 99.92
Skipping NRB, due to only have 38/50 years of data
Skipping NRN, due to only have 35/50 years of data
Skipping NRS, due to only have 19/50 years of data
NSE d2022:57 #28.0, climate: 60.18
Skipping NSI, due to only have 30/50 years of data
NTD d2022:0.0 #24.0, climate: 0.1
NTU d2022:40.0 #36.5, climate: 29.74
Skipping NUC, due to only have 18/50 years of data
NUQ d2022:0.0 #23.0, climate: 0.44
NUW d2022:0.0 #24.5, climate: 0.06
Skipping NXX, due to only have 31/50 years of data
Skipping NYC, due to only have 30/50 years of data
NYG d2022:7.0 #10.5, climate: 28.08
NZY d2022:0 #25.5, climate: 0.0
Skipping O87, due to only have 0/50 years of data
OAK d2

Skipping SRN, due to only have 2/50 years of data
SRQ d2022:99.0 #29.0, climate: 100.32
SSC d2022:72.0 #40.0, climate: 43.74
SSI d2022:87 #41.0, climate: 63.46
STC d2022:14.0 #44.5, climate: 5.54
Skipping STJ, due to only have 37/50 years of data
STK d2022:2.0 #47.5, climate: 0.66
STL d2022:118 #47.0, climate: 50.22
Skipping STP, due to only have 30/50 years of data
STS d2022:0.0 #14.0, climate: 3.06
Skipping SUN, due to only have 36/50 years of data
Skipping SUS, due to only have 36/50 years of data
SUU d2022:3.0 #23.0, climate: 6.48
SUX d2022:35.0 #39.0, climate: 25.36
Skipping SVC, due to only have 22/50 years of data
SVN d2022:166.0 #43.0, climate: 92.38
SWF d2022:0.0 #10.5, climate: 5.28
Skipping SWO, due to only have 25/50 years of data
SXT d2022:0 #25.5, climate: 0.0
Skipping SYF, due to only have 39/50 years of data
SYR d2022:0.0 #13.5, climate: 3.5
SZL d2022:91.0 #44.5, climate: 45.1
TAD d2022:0.0 #25.0, climate: 0.02
Skipping TAN, due to only have 24/50 years of data
TBN d202

In [2]:
df.to_csv('/tmp/saveme.csv')
#df['ratio'] = df.val / df.climate

In [9]:
# df[df.network=='OK_ASOS'].sort(['ratio'])
df

,sid,lat,lon,d2022,climate,network,d2022_rank
0,AAF,29.727610,-85.027440,101.0,59.40,FL_ASOS,40.0
1,ABE,40.650830,-75.449170,0.0,6.64,PA_ASOS,10.0
2,ABI,32.410629,-99.682086,191.0,32.94,TX_ASOS,50.0
3,ABQ,35.041900,-106.615500,0.0,0.02,NM_ASOS,25.0
4,ABR,45.450000,-98.420000,26.0,10.66,SD_ASOS,47.0
...,...,...,...,...,...,...,...
631,WRL,43.965710,-107.950830,0.0,0.82,WY_ASOS,19.5
632,YKM,46.568170,-120.544060,0.0,2.38,WA_ASOS,20.5
633,YKN,42.916690,-97.385940,37.0,18.78,SD_ASOS,43.0
634,YNG,41.254440,-80.673890,10.0,1.50,OH_ASOS,48.5


In [30]:
#df = pd.DataFrame(rows)
#df2 = df[df.ratio<10]

from pyiem.plot import MapPlot, get_cmap
import numpy as np
df['delta'] = df['d2022'] - df['climate']
df['rank2'] = 50 - df['d2022_rank']
#print(df['delta'].describe())

mp = MapPlot(
    sector='conus',
    title=r"Departure of 2022 Hours with Heat Index >= 100$^\circ$F vs 1973-2022 Climatology",
            subtitle="NOAA hourly METARs through 20 July. X denotes 2022 value is maximum since 1973.")
cmap = get_cmap('RdBu_r')
# cmap.set_under('white')
cmap.set_bad('white')
mp.contourf(np.array(df.lon), np.array(df.lat), df['delta'],
           [-100, -50, -25, -5, 5, 25, 50, 100], 
           units='Hours', cmap=cmap)
df2 = df[(df['rank2'] == 0)]
mp.plot_values(df2.lon, df2.lat, ['X'] * len(df2.index), fmt='%s', labelbuffer=0)
mp.postprocess(filename='conus.png')
mp.close()